In [10]:
# Disable widget-based progress bars
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # Add this!

# Use simple progress bars instead of widgets
import transformers
transformers.logging.set_verbosity_error()

print("✓ Environment configured")

✓ Environment configured


### Clear GPU Memory

In [11]:
import torch
import gc

# Force cleanup
gc.collect()
torch.cuda.empty_cache()

# Check memory
print(f"GPU Memory available: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB total")
print(f"GPU Memory allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
print(f"GPU Memory reserved: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")

GPU Memory available: 94.97 GB total
GPU Memory allocated: 0.00 GB
GPU Memory reserved: 0.00 GB


## Load the model

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch

# # Point to local model directory
# model_id = "./gpt-oss-20b/"

# # Load model and tokenizer
# print("Loading model with MXFP4 quantization...")
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype="auto",
#     device_map="auto",
#     trust_remote_code=True,
#     local_files_only=True,
#     low_cpu_mem_usage=True,
# )

# print("Loading tokenizer...")
# tokenizer = AutoTokenizer.from_pretrained(
#     model_id,
#     local_files_only=True
# )

# print("✓ Model loaded successfully!")
# print(f"GPU Memory allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

Loading model with MXFP4 quantization...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading tokenizer...
✓ Model loaded successfully!
GPU Memory allocated: 38.96 GB


## Run inference


In [ ]:
# from transformers import TextStreamer
# # Test inference
# messages = [
#     {"role": "user", "content": "What is 2+2?"}
# ]

# # Apply chat template (harmony format)
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True
# )

# inputs = tokenizer(text, return_tensors="pt").to(model.device)

# # Generate
# print("Generating...")


# # Use streamer to see tokens as they're generated
# streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# outputs = model.generate(
#     **inputs,
#     max_new_tokens=256,
#     do_sample=False,
#     streamer=streamer  # Add this
# )

# response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
# print("\nResponse:", response)

Generating...
analysisThe user asks: "What is 2+2?" It's a simple arithmetic question. The answer is 4. We should respond with the answer. Possibly also mention that it's 4. The user didn't ask for anything else. So answer: 4.assistantfinal4

Response: analysisThe user asks: "What is 2+2?" It's a simple arithmetic question. The answer is 4. We should respond with the answer. Possibly also mention that it's 4. The user didn't ask for anything else. So answer: 4.assistantfinal4


# Steering


In [1]:
import logging
import os
import sys
import timez
from collections import defaultdict
from pathlib import Path

import circuitsvis as cv
import einops
import numpy as np
import torch as t
from IPython.display import display
from jaxtyping import Float
from nnsight import CONFIG, LanguageModel
from openai import OpenAI
from rich import print as rprint
from rich.table import Table
from torch import Tensor

# Hide some info logging messages from nnsight
logging.disable(sys.maxsize)

t.set_grad_enabled(False)
device = t.device(
    "mps" if t.backends.mps.is_available() else "cuda" if t.cuda.is_available() else "cpu"
)


from plotly_utils import imshow

MAIN = __name__ == "__main__"

In [2]:
model = LanguageModel("./gpt-oss-20b", device_map="auto", torch_dtype=t.bfloat16)
tokenizer = model.tokenizer

print()

N_HEADS = model.config.num_attention_heads
N_LAYERS = model.config.num_hidden_layers
D_MODEL = model.config.hidden_size
D_HEAD = model.config.head_dim

print(f"Number of heads: {N_HEADS}")
print(f"Number of layers: {N_LAYERS}")
print(f"Model (hidden) dimension: {D_MODEL}")
print(f"Head dimension: {D_HEAD}\n")

print("Entire config: ", model.config)


Number of heads: 64
Number of layers: 24
Model (hidden) dimension: 2880
Head dimension: 64

Entire config:  GptOssConfig {
  "architectures": [
    "GptOssForCausalLM"
  ],
  "attention_bias": true,
  "attention_dropout": 0.0,
  "dtype": "bfloat16",
  "eos_token_id": 200002,
  "experts_per_token": 4,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2880,
  "initial_context_length": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 2880,
  "layer_types": [
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    

### Test tokenizer

In [3]:
print(tokenizer("This must be Thursday", return_tensors="pt"))

print(tokenizer.decode([ 2500,  2804,   413, 12453]))


{'input_ids': tensor([[ 2500,  2804,   413, 12453]]), 'attention_mask': tensor([[1, 1, 1, 1]])}
This must be Thursday


## Access Hidden States

### GPT-OSS-20B Quick Reference

- 24 layers | 2880 hidden | 64 heads | 201K vocab
- **Key difference**: `model.model.layers[i]` not `model.transformer.h[i]`

### Basic Usage
```python
with model.trace(prompt, remote=False):
    # Hidden states from layer i
    hidden = model.model.layers[i].output[0].save()
    
    # Logits (last token)
    logits = model.lm_head.output[0, -1].save()
```

### Steering
```python
with model.trace(prompt, remote=False):
    hidden = model.model.layers[i].output[0]
    hidden[:, -1, :] += steering_vector
    logits = model.lm_head.output[0, -1].save()
```

### Multi-Layer Extraction
```python
with model.trace(prompt, remote=False):
    all_hidden = [model.model.layers[i].output[0].save() for i in range(24)]
```

## Layer Components
```python
model.model.layers[i].self_attn     # Attention block
model.model.layers[i].mlp           # MoE block
model.model.embed_tokens            # Embeddings
model.model.norm                    # Final norm
model.lm_head                       # Output layer
```

## Expected Shapes

- Hidden: `(1, seq_len, 2880)`
- Logits: `(201088,)` for single token

In [4]:
# If you have an API key & want to work remotely, then set REMOTE = True and replace "YOUR-API-KEY"
# with your actual key. If not, then leave REMOTE = False.
REMOTE = False
prompt = "The Eiffel Tower is in the city of"

with model.trace(prompt, remote=REMOTE):
    # Save the model's hidden states
    hidden_states = model.model.layers[-1].output[0].save()

    # Save the model's logit output
    logits = model.lm_head.output[0, -1].save()

# Get the model's logit output, and it's next token prediction
print(f"logits.shape = {logits.shape} = (vocab_size,)")
print("Predicted token ID =", predicted_token_id := logits.argmax().item())
print(f"Predicted token = {tokenizer.decode(predicted_token_id)!r}")

# Print the shape of the model's residual stream
print(f"\nresid.shape = {hidden_states.shape} = (batch_size, seq_len, d_model)")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

logits.shape = torch.Size([201088]) = (vocab_size,)
Predicted token ID = 12650
Predicted token = ' Paris'

resid.shape = torch.Size([8, 2880]) = (batch_size, seq_len, d_model)


# Datasets


## Load ToM Datasets

Instead of using the ICL dataset, we'll load two Theory of Mind (ToM) datasets:
1. **ToM Dataset** (`first_order_1_tom_prompts.jsonl`) - Questions that require theory of mind reasoning
2. **No-ToM Dataset** (`first_order_1_no_tom_prompts.jsonl`) - Questions that don't require theory of mind


In [10]:
import json
from pathlib import Path

class ToMDataset:
    """
    Dataset for Theory of Mind (ToM) tasks from JSONL files.
    
    Inputs:
        jsonl_path: Path to the JSONL file containing prompts
        size: Optional limit on number of examples to load (None = load all)
    """
    
    def __init__(self, jsonl_path: str, size: int = None):
        self.jsonl_path = Path(jsonl_path)
        self.data = []
        self.prompts = []
        self.completions = []
        self.answers = []
        self.question_types = []
        self.story_types = []
        self.requires_tom = []
        
        # Load data from JSONL file
        with open(self.jsonl_path, 'r') as f:
            for i, line in enumerate(f):
                if size is not None and i >= size:
                    break
                item = json.loads(line)
                self.data.append(item)
                self.prompts.append(item['prompt'])
                # Completions are the answers with a leading space
                self.completions.append(' ' + item['answer'])
                self.answers.append(item['answer'])
                self.question_types.append(item['question_type'])
                self.story_types.append(item['story_type'])
                self.requires_tom.append(item['requires_tom'])
        
        self.size = len(self.data)
    
    def __len__(self):
        return self.size
    
    def __getitem__(self, idx: int):
        return self.data[idx]
    
    def __repr__(self):
        return f"ToMDataset(path={self.jsonl_path.name}, size={self.size}, requires_tom={self.requires_tom[0] if self.size > 0 else 'N/A'})"


# Load the two datasets
tom_dataset = ToMDataset(
    "tom_benchmarks/tomi/tomi_pairs/first_order_1_tom_prompts.jsonl"
)
no_tom_dataset = ToMDataset(
    "tom_benchmarks/tomi/tomi_pairs/first_order_1_no_tom_prompts.jsonl"
)

print("ToM Dataset (requires ToM):")
print(f"  {tom_dataset}")
print(f"  Example prompt:\n{tom_dataset.prompts[0][:200]}...")
print(f"  Example answer: {tom_dataset.answers[0]}")

print("\nNo-ToM Dataset (doesn't require ToM):")
print(f"  {no_tom_dataset}")
print(f"  Example prompt:\n{no_tom_dataset.prompts[0][:200]}...")
print(f"  Example answer: {no_tom_dataset.answers[0]}")


ToM Dataset (requires ToM):
  ToMDataset(path=first_order_1_tom_prompts.jsonl, size=404, requires_tom=True)
  Example prompt:
Story:
Isabella entered the den.
Olivia entered the den.
Isabella dislikes the pumpkin
The broccoli is in the blue_pantry.
Isabella exited the den.
Olivia moved the broccoli to the red_drawer.
Abigail...
  Example answer: blue_pantry

No-ToM Dataset (doesn't require ToM):
  ToMDataset(path=first_order_1_no_tom_prompts.jsonl, size=595, requires_tom=False)
  Example prompt:
Story:
Aria entered the front_yard.
Aiden entered the front_yard.
The grapefruit is in the green_bucket.
Aria moved the grapefruit to the blue_container.
Aiden exited the front_yard.
Noah entered the ...
  Example answer: blue_container


### Usage Notes

The `ToMDataset` class has the same interface as `ICLDataset`, so it can be used interchangeably:
- `.prompts` - list of prompt strings
- `.completions` - list of completion strings (with leading space)
- `.answers` - list of answer strings
- `.size` - number of examples

**Key difference from ICLDataset**: 
- ToMDataset does **not** have a `create_corrupted_dataset()` method
- If you need corrupted data for contrastive analysis, you'll need to implement that separately


In [11]:
# Example: View a complete example from each dataset
print("=" * 80)
print("EXAMPLE FROM TOM DATASET (requires ToM reasoning):")
print("=" * 80)
print(tom_dataset[0]['prompt'])
print(f"Correct answer: {tom_dataset[0]['answer']}")
print(f"Story type: {tom_dataset[0]['story_type']}")
print(f"Requires ToM: {tom_dataset[0]['requires_tom']}")

print("\n" + "=" * 80)
print("EXAMPLE FROM NO-TOM DATASET (doesn't require ToM):")
print("=" * 80)
print(no_tom_dataset[0]['prompt'])
print(f"Correct answer: {no_tom_dataset[0]['answer']}")
print(f"Story type: {no_tom_dataset[0]['story_type']}")
print(f"Requires ToM: {no_tom_dataset[0]['requires_tom']}")


EXAMPLE FROM TOM DATASET (requires ToM reasoning):
Story:
Isabella entered the den.
Olivia entered the den.
Isabella dislikes the pumpkin
The broccoli is in the blue_pantry.
Isabella exited the den.
Olivia moved the broccoli to the red_drawer.
Abigail entered the garden.
Isabella entered the garden.

Question: Where will Isabella look for the broccoli?
Answer:
Correct answer: blue_pantry
Story type: false_belief
Requires ToM: True

EXAMPLE FROM NO-TOM DATASET (doesn't require ToM):
Story:
Aria entered the front_yard.
Aiden entered the front_yard.
The grapefruit is in the green_bucket.
Aria moved the grapefruit to the blue_container.
Aiden exited the front_yard.
Noah entered the playroom.

Question: Where will Aiden look for the grapefruit?
Answer:
Correct answer: blue_container
Story type: true_belief
Requires ToM: False


In [29]:
prompt = """Isabella entered the den.
Olivia entered the den.
Isabella dislikes the pumpkin
The broccoli is in the blue_pantry.
Isabella exited the den.
Olivia moved the broccoli to the red_drawer.
Abigail entered the garden.
Isabella entered the garden.

Question: Where will Isabella look for the broccoli?
Answer:
"""

# Tokenize the prompt and move to GPU
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Use nnsight's generate method (pass tokenized inputs)
output = model.generate(
    inputs["input_ids"],
    max_new_tokens=500,
    do_sample=False,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)


# Or decode just the new tokens
new_tokens = tokenizer.decode(output[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
print("\nGenerated text only:", new_tokens)


Generated text only: We have a story with characters: Isabella, Olivia, Abigail. Items: broccoli, pumpkin, blue_pantry, red_drawer. The question: "Where will Isabella look for the broccoli?" We need to infer from the story. Let's parse the story:

- Isabella entered the den.
- Olivia entered the den.
- Isabella dislikes the pumpkin.
- The broccoli is in the blue_pantry.
- Isabella exited the den.
- Olivia moved the broccoli to the red_drawer.
- Abigail entered the garden.
- Isabella entered the garden.

We need to answer: Where will Isabella look for the broccoli? The story says the broccoli was originally in the blue_pantry. Then Olivia moved the broccoli to the red_drawer. So the broccoli is now in the red_drawer. Isabella is in the garden. So where will she look? She will look in the red_drawer. But maybe she will look in the garden? But the broccoli is not in the garden. The question: "Where will Isabella look for the broccoli?" The answer: She will look in the red_drawer. But may

## Calculate Function Vector

In [30]:

def calculate_fn_vector(
    model: model,
    dataset: no_tom_dataset,
    head_list: list[tuple[int, int]],
) -> Float[Tensor, "D_MODEL"]:
    """
    Returns a vector of length `D_MODEL`, containing the sum of vectors written to the residual
    stream by the attention heads in `head_list`, averaged over all inputs in `dataset`.

    Inputs:
        model: LanguageModel
            the transformer you're doing this computation with
        dataset: ICLDataset
            the dataset of clean prompts from which we'll extract the function vector (we'll also
            create a corrupted version of this dataset for interventions)
        head_list: list[tuple[int, int]]
            list of attention heads we're calculating the function vector from
    """
    # Turn head_list into a dict of {layer: heads we need in this layer}
    head_dict = defaultdict(set)
    for layer, head in head_list:
        head_dict[layer].add(head)

    fn_vector_list = []

    with model.trace(dataset.prompts, remote=REMOTE):
        for layer, head_list in head_dict.items():
            # Get the output projection layer
            out_proj = model.transformer.h[layer].attn.out_proj

            # Get the mean output projection input (note, setting values of this tensor will not
            # have downstream effects on other tensors)
            hidden_states = out_proj.input[:, -1].mean(dim=0)

            # Zero-ablate all heads which aren't in our list, then get the output (which
            # will be the sum over the heads we actually do want!)
            heads_to_ablate = set(range(N_HEADS)) - head_dict[layer]
            for head in heads_to_ablate:
                hidden_states.reshape(N_HEADS, D_HEAD)[head] = 0.0

            # Now that we've zeroed all unimportant heads, get the output & add it to the list
            # (we need a single batch dimension so we can use `out_proj`)
            out_proj_output = out_proj(hidden_states.unsqueeze(0)).squeeze()
            fn_vector_list.append(out_proj_output.save())

    # We sum all attention head outputs to get our function vector
    fn_vector = sum([v for v in fn_vector_list])

    assert fn_vector.shape == (D_MODEL,)
    return fn_vector

In [ ]:
def intervene_with_fn_vector(
    model: LanguageModel,
    word: str,
    layer: int,
    fn_vector: Float[Tensor, "d_model"],
    prompt_template='The word "{x}" means',
    n_tokens: int = 5,
) -> tuple[str, str]:
    """
    Intervenes with a function vector, by adding it at the last sequence position of a generated
    prompt.

    Inputs:
        model: LanguageModel
            the transformer you're doing this computation with
        word: str
            The word substituted into the prompt template, via prompt_template.format(x=word)
        layer: int
            The layer we'll make the intervention (by adding the function vector)
        fn_vector: Float[Tensor, "d_model"]
            The vector we'll add to the final sequence position for each new token to be generated
        prompt_template:
            The template of the prompt we'll use to produce completions
        n_tokens: int
            The number of additional tokens we'll generate for our unsteered / steered completions

    Returns:
        completion: str
            The full completion (including original prompt) for the no-intervention case
        completion_intervention: str
            The full completion (including original prompt) for the intervention case
    """
    prompt = prompt_template.format(x=word)

    with model.generate(
        remote=REMOTE, max_new_tokens=n_tokens, repetition_penalty=1.2
    ) as generator:
        with model.all():
            with generator.invoke(prompt):
                tokens = model.generator.output.save()

            with generator.invoke(prompt):
                model.transformer.h[layer].output[0][:, -1] += fn_vector
                tokens_intervention = model.generator.output.save()

    completion, completion_intervention = tokenizer.batch_decode(
        [tokens.squeeze().tolist(), tokens_intervention.squeeze().tolist()]
    )
    return completion, completion_intervention